In [50]:
import pandas as pd
import os
import re
import numpy as np
import datetime
# import matplotlib.pyplot as plt
# from IPython.core.interactiveshell import InteractiveShell

In [2]:
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows",None)
# pd.set_option("display.mpl_style", "default")
# pd.set_option("display.line_width", 5000)
pd.set_option("display.expand_frame_repr", False)
# InteractiveShell.ast_node_interactivity = "all"
# %matplotlib inline

In [3]:
file_path = "C:\\Users\\Jinghui\\Desktop\\DECA\\condaenvd\\spider&analysis\\sotdata\\"
files_list = os.listdir(file_path)
files_list
# file = pd.ExcelFile(file_path + files_list[-1])
# file.sheet_names

['SOT-LT-PPM-SHIP_W01.xlsx',
 'SOT-LT-PPM-SHIP_W48.xlsx',
 'SOT-LT-PPM-SHIP_W49.xlsx',
 'SOT-LT-PPM-SHIP_W50.xlsx',
 'SOT-LT-PPM-SHIP_W51.xlsx',
 'SOT-LT-PPM-SHIP_W52.xlsx']

In [113]:
sheetname = "ship007"
index_col = "DATE_DEP_COLIS"
# index_col = "DEP_DATE"
f_l = []
for i in files_list:
    f = pd.read_excel(file_path + i, sheet_name=sheetname)
    f_l.append(f)
df = pd.concat(f_l)

In [121]:
df["DATE_DEP_COLIS"].value_counts()

DATE_DEP_COLIS
20241210    16481
20241213    11007
20241212    10124
20241127     9161
20241211     8436
20241230     8270
20241214     7988
20241201     7319
20241128     7208
20250104     7155
20241125     6919
20250102     6885
20241221     6659
20241215     6539
20241130     6533
20241202     6499
20241217     6318
20241126     6229
20241129     6139
20241223     6042
20241222     5781
20250103     5727
20250101     5723
20241218     5673
20241203     5632
20241207     5546
20250105     5448
20241209     5439
20241220     5363
20241228     5303
20241204     5282
20241205     5189
20241216     4994
20241226     4971
20241219     4917
20241229     4693
20241224     4685
20241208     4643
20241225     4605
20241206     4575
20241231     4335
20241227     4276
Name: count, dtype: int64

In [122]:
# Total LT 大于18取值
df["18H Ontime Qty"] = df.apply(lambda x: 0 if x["Total LT"]>18.0 else x.QTES,axis=1)
df["15H Ontime Qty"] = df.apply(lambda x: 0 if x["Total LT"]>15.0 else x.QTES,axis=1)
df["24H Ontime Qty"] = df.apply(lambda x: 0 if x["Total LT"]>24.0 else x.QTES,axis=1)

In [123]:
df1 = df[["DATE_DEP_COLIS","Total LT","Ontime Qty","18H Ontime Qty","15H Ontime Qty","Mono/809","QTES"]]
# 通过index 按条件取值，将日期列恢复为正常列
# df = df[df.index >= "20241201"]
# df = df[df.index < "20250101"]
# df = df.reset_index()
df1["DATE_DEP_COLIS"] = df1["DATE_DEP_COLIS"].astype("str")
df1["DATE_DEP_COLIS"] = pd.to_datetime(df1["DATE_DEP_COLIS"])
df1 = df1[df1["DATE_DEP_COLIS"] >= datetime.datetime(2024,12,1)]
df1 = df1[df1["DATE_DEP_COLIS"] < datetime.datetime(2025,1,1)]

C:\Users\Jinghui\AppData\Local\Temp\ipykernel_24316\3853432575.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["DATE_DEP_COLIS"] = df1["DATE_DEP_COLIS"].astype("str")
C:\Users\Jinghui\AppData\Local\Temp\ipykernel_24316\3853432575.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["DATE_DEP_COLIS"] = pd.to_datetime(df1["DATE_DEP_COLIS"])


In [124]:
df11 = df1.groupby(["Mono/809","DATE_DEP_COLIS"]).agg({
                        "Total LT":np.mean,                        
                        "Ontime Qty":np.sum,
                        "15H Ontime Qty":np.sum, 
                        "18H Ontime Qty":np.sum,                        
                        "QTES":np.sum
                            })

In [125]:
df11["12H SOT"] = (df11["Ontime Qty"]/df11["QTES"]).map(lambda x:format(x,".2%"))
df11["15H SOT"] = (df11["15H Ontime Qty"]/df11["QTES"]).map(lambda x:format(x,".2%"))
df11["18H SOT"] = (df11["18H Ontime Qty"]/df11["QTES"]).map(lambda x:format(x,".2%"))
df11.to_csv("./datatemp/1.csv")

In [62]:
# 分三类订单
df_mono = df[df["Mono/809"] == "Mono"]
df_shortage = df[df["Mono/809"] == "Shortage"]
df_809 = df[df["Mono/809"] == 809]

In [24]:
# df_809.info()
# df_mono.info()
# df_shortage.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21749 entries, 9 to 163000
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   DATE_DEP_COLIS  21749 non-null  int64  
 1   QTES            21749 non-null  int64  
 2   Total LT        21749 non-null  float64
 3   Ontime Qty      21749 non-null  int64  
 4   18H Ontime Qty  21749 non-null  int64  
 5   15H Ontime Qty  21749 non-null  int64  
 6   Mono/809        21749 non-null  object 
 7   24H Ontime Qty  21749 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 1.5+ MB


In [63]:
# 按日期透视，对不同列进行不同聚合
mono_pivot = df_mono.pivot_table(index="DATE_DEP_COLIS", values=["QTES","18H Ontime Qty","15H Ontime Qty","Ontime Qty","Total LT","24H Ontime Qty"], \
                    aggfunc={\
                        "18H Ontime Qty":np.sum,\
                        "15H Ontime Qty":np.sum,\
                        "24H Ontime Qty":np.sum,\
                        "Ontime Qty":np.sum,\
                        "QTES":np.sum,\
                        "Total LT":np.mean\
                            }).reset_index()
shortage_pivot = df_shortage.pivot_table(index="DATE_DEP_COLIS", values=["QTES","18H Ontime Qty","15H Ontime Qty","Ontime Qty","Total LT","24H Ontime Qty"], \
                    aggfunc={\
                        "18H Ontime Qty":np.sum,\
                        "15H Ontime Qty":np.sum,\
                        "24H Ontime Qty":np.sum,\
                        "Ontime Qty":np.sum,\
                        "QTES":np.sum,\
                        "Total LT":np.mean\
                            }).reset_index()
pivot_809 = df_809.pivot_table(index="DATE_DEP_COLIS", values=["QTES","18H Ontime Qty","15H Ontime Qty","Ontime Qty","Total LT","24H Ontime Qty"], \
                    aggfunc={\
                        "18H Ontime Qty":np.sum,\
                        "15H Ontime Qty":np.sum,\
                        "24H Ontime Qty":np.sum,\
                        "Ontime Qty":np.sum,\
                        "QTES":np.sum,\
                        "Total LT":np.mean\
                            }).reset_index()

# mono_pivot

In [64]:
pivot_809

,DATE_DEP_COLIS,15H Ontime Qty,18H Ontime Qty,24H Ontime Qty,Ontime Qty,QTES,Total LT
0,20241201,11089,11163,11163,10228,11163,6.259234
1,20241202,10616,10616,10617,10615,10617,6.575076
2,20241203,5451,5454,5506,5451,5506,5.421044
3,20241204,6091,6091,6091,6071,6091,6.198864
4,20241205,6466,6466,6466,6466,6466,5.961254
5,20241206,5877,5877,5877,5877,5877,5.848106
6,20241207,11132,11132,11132,11131,11132,6.000210
7,20241208,6474,6474,6474,6474,6474,6.061616
8,20241209,8706,8707,8712,8680,8712,7.021153
9,20241210,20666,22397,22759,19616,22762,6.783712


In [30]:
# 计算SOT的值； 通过loc方法建 汇总行，用不同聚合得到结果
mono_pivot["18H SOT"] = (mono_pivot["18H Ontime Qty"] / mono_pivot["QTES"])
mono_pivot["15H SOT"] = (mono_pivot["15H Ontime Qty"] / mono_pivot["QTES"])
mono_pivot["12H SOT"] = (mono_pivot["Ontime Qty"] / mono_pivot["QTES"])
mono_pivot["24H SOT"] = (mono_pivot["24H Ontime Qty"] / mono_pivot["QTES"])
mono_pivot.loc["汇总"] = mono_pivot.agg({\
                                        "18H Ontime Qty":np.sum,\
                                        "15H Ontime Qty":np.sum,\
                                        "Ontime Qty":np.sum,\
                                        "24H Ontime Qty":np.sum,\
                                        "QTES":np.sum,\
                                        "Total LT":np.mean,\
                                        "18H SOT":np.mean,\
                                        "15H SOT":np.mean,\
                                        "12H SOT":np.mean,\
                                        "24H SOT":np.mean,\
                                        })
mono_pivot["18H SOT"] = mono_pivot["18H SOT"].map(lambda x:format(x,".2%"))
mono_pivot["15H SOT"] = mono_pivot["15H SOT"].map(lambda x:format(x,".2%"))
mono_pivot["12H SOT"] = mono_pivot["12H SOT"].map(lambda x:format(x,".2%"))
mono_pivot["24H SOT"] = mono_pivot["24H SOT"].map(lambda x:format(x,".2%"))
mono_pivot[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]] = mono_pivot[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]].astype("int")
mono_pivot[index_col] = pd.to_datetime(mono_pivot[index_col])

shortage_pivot["18H SOT"] = (shortage_pivot["18H Ontime Qty"] / shortage_pivot["QTES"])
shortage_pivot["15H SOT"] = (shortage_pivot["15H Ontime Qty"] / shortage_pivot["QTES"])
shortage_pivot["12H SOT"] = (shortage_pivot["Ontime Qty"] / shortage_pivot["QTES"])
shortage_pivot["24H SOT"] = (shortage_pivot["24H Ontime Qty"] / shortage_pivot["QTES"])
shortage_pivot.loc["汇总"] = shortage_pivot.agg({\
                                        "18H Ontime Qty":np.sum,\
                                        "15H Ontime Qty":np.sum,\
                                        "24H Ontime Qty":np.sum,\
                                        "Ontime Qty":np.sum,\
                                        "QTES":np.sum,\
                                        "Total LT":np.mean,\
                                        "18H SOT":np.mean,\
                                        "15H SOT":np.mean,\
                                        "12H SOT":np.mean,\
                                        "24H SOT":np.mean,\
                                        })
shortage_pivot["18H SOT"] = shortage_pivot["18H SOT"].map(lambda x:format(x,".2%"))
shortage_pivot["15H SOT"] = shortage_pivot["15H SOT"].map(lambda x:format(x,".2%"))
shortage_pivot["12H SOT"] = shortage_pivot["12H SOT"].map(lambda x:format(x,".2%"))
shortage_pivot["24H SOT"] = shortage_pivot["24H SOT"].map(lambda x:format(x,".2%"))
shortage_pivot[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]] = shortage_pivot[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]].astype("int")
shortage_pivot[index_col] = pd.to_datetime(shortage_pivot[index_col])

pivot_809["18H SOT"] = (pivot_809["18H Ontime Qty"] / pivot_809["QTES"])
pivot_809["15H SOT"] = (pivot_809["15H Ontime Qty"] / pivot_809["QTES"])
pivot_809["12H SOT"] = (pivot_809["Ontime Qty"] / pivot_809["QTES"])
pivot_809["24H SOT"] = (pivot_809["24H Ontime Qty"] / pivot_809["QTES"])
pivot_809.loc["汇总"] = pivot_809.agg({\
                                        "18H Ontime Qty":np.sum,\
                                        "15H Ontime Qty":np.sum,\
                                        "24H Ontime Qty":np.sum,\
                                        "Ontime Qty":np.sum,\
                                        "QTES":np.sum,\
                                        "Total LT":np.mean,\
                                        "18H SOT":np.mean,\
                                        "15H SOT":np.mean,\
                                        "12H SOT":np.mean,\
                                        "24H SOT":np.mean,\
                                        })
pivot_809["18H SOT"] = pivot_809["18H SOT"].map(lambda x:format(x,".2%"))
pivot_809["15H SOT"] = pivot_809["15H SOT"].map(lambda x:format(x,".2%"))
pivot_809["12H SOT"] = pivot_809["12H SOT"].map(lambda x:format(x,".2%"))
pivot_809["24H SOT"] = pivot_809["24H SOT"].map(lambda x:format(x,".2%"))
pivot_809[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]] = pivot_809[["18H Ontime Qty","15H Ontime Qty","24H Ontime Qty","Ontime Qty","QTES"]].astype("int")
pivot_809[index_col] = pd.to_datetime(pivot_809[index_col])
# mono_pivot

In [31]:
mono_pivot.to_csv("./datatemp/Mono.csv",index=False)
shortage_pivot.to_csv("./datatemp/hortage.csv",index=False)
pivot_809.to_csv("./datatemp/809.csv",index=False)